In [11]:
import zipfile
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [12]:
path = 'archive.zip'

In [13]:
with zipfile.ZipFile(path,'r') as zipfiles:
    zipfiles.extractall('extracted_files')

In [14]:
train_folder = os.path.join('extracted_files','train')

In [15]:
train_sub = os.listdir(train_folder)
print (train_sub)

['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two of hearts', 'two of spades']


In [16]:
data = pd.read_csv('cards.csv')

In [17]:
data = data.drop(columns=['filepaths','data set'])

In [18]:
print (data.head(5))

   class index        labels card type
0            0  ace of clubs       ace
1            0  ace of clubs       ace
2            0  ace of clubs       ace
3            0  ace of clubs       ace
4            0  ace of clubs       ace


In [19]:
data_generator = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [20]:
train_image_generator = data_generator.flow_from_directory(train_folder,target_size=(200,200),batch_size=4,class_mode = 'categorical',subset='training')
validation_image_generator = data_generator.flow_from_directory(train_folder,target_size=(200,200),batch_size=4,class_mode = 'categorical',subset = 'validation')

Found 6121 images belonging to 53 classes.
Found 1503 images belonging to 53 classes.


In [29]:
model = Sequential([Conv2D(32,(3,3),activation='relu',input_shape=(200,200,3)),
                    MaxPooling2D(2,2),
                    Dropout(0.075),
                   Conv2D(64,(3,3),activation='relu'),
                   MaxPooling2D((2,2)),
                    Dropout(0.075),
                   Conv2D(128,(3,3),activation='relu'),
                   MaxPooling2D((2,2)),
                    Dropout(0.075),
                   Conv2D(256,(3,3),activation='relu'),
                   MaxPooling2D((2,2)),
                    Dropout(0.075),
                   Flatten(),
                   Dense(512,activation='relu'),
                    Dropout(0.25),
                    Dense(train_image_generator.num_classes,activation='softmax')])

In [30]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 99, 99, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 48, 48, 64)       0         
 2D)                                                             
                                                                 
 dropout_11 (Dropout)        (None, 48, 48, 64)       

In [32]:
model.fit(train_image_generator,steps_per_epoch=train_image_generator.samples // train_image_generator.batch_size,
         epochs = 5,
         validation_data = validation_image_generator,
          validation_steps=validation_image_generator.samples // validation_image_generator.batch_size)

Epoch 1/5
1530/1530 [==============================] - 24s 15ms/step - loss: 3.7752 - accuracy: 0.0562 - val_loss: 2.5559 - val_accuracy: 0.3047
Epoch 2/5
1530/1530 [==============================] - 23s 15ms/step - loss: 2.2692 - accuracy: 0.3874 - val_loss: 1.6318 - val_accuracy: 0.5753
Epoch 3/5
1530/1530 [==============================] - 22s 15ms/step - loss: 1.5640 - accuracy: 0.5772 - val_loss: 1.1469 - val_accuracy: 0.6933
Epoch 4/5
1530/1530 [==============================] - 23s 15ms/step - loss: 1.1764 - accuracy: 0.6711 - val_loss: 1.0855 - val_accuracy: 0.7267
Epoch 5/5
1530/1530 [==============================] - 23s 15ms/step - loss: 0.8781 - accuracy: 0.7510 - val_loss: 1.1494 - val_accuracy: 0.7393
